# Лекция 2: Обучение искуственных нейронных сетей в PyTorch

__Автор: Сергей Вячеславович Макрушин__ e-mail: SVMakrushin@fa.ru 

Финансовый универсиет, 2021 г. 

При подготовке лекции использованы материалы:
* ...

* v0.5 старое название: TCN20_lNNp2_2_v5
* v0.6 18.02.2021 
* v0.7 22.02.2021 

## Разделы: <a class="anchor" id="разделы"></a>
* [Современные методы обучения нейронной сети и обратное распространение ошибки](#современные-методы)
* [Обучение модели нейронной сети, алгоритм обратного распространения ошибки](#обратное-распространение)
    * [Проблема обучения модели нейронной сети](#проблема-обучения)
    * [Проблема поиска градиента](#проблема-поиска)
* [Дифференцируемое программирование и реализация обратного распространения ошибки](#дифференцируемое)
    * [Автоматическое дифференциирование в PyTorch](#автоматическое-PyTorch)

-

* [к оглавлению](#разделы)

In [26]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v2.css")
HTML(html.read().decode('utf-8'))

## Современные методы обучения нейронной сети и обратное распространение ошибки <a class="anchor" id="современные-методы"></a>
* [к оглавлению](#разделы)

----
### Применение тензоров:  прямое распространение сигналов и оценка ошибки

__Постановка задачи__ 

* У нас есть набор данных $D$, состоящий из пар $(\pmb{x}, \pmb{y})$, где $\pmb{x}$ - признаки, а $\pmb{y}$ - правильный ответ. 
* Модель сети $f_L$, имеющей $L$ слоев с весами $\pmb{\theta}$ (совокупность весов нейронов из всех слоев) на этих данных делает некоторые предсказания $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})$
* Задана функция ошибки $E$, которую можно подсчитать на каждом примере: $E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$ (например, это может быть квадрат или модуль отклонения $\hat{\pmb{y}}$ от $\pmb{y}$ в случае регрессии или перекрестная энтропия в случае классификации)
* Тогда суммарная ошибка на наборе данных $D$ будет функцией от параметров модели: $E(\pmb{\theta})$ и определяется как $E(\pmb{\theta})=\sum_{(\pmb{x}, \pmb{y}) \in D} E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 600px;"/><br/>
    <b>Прямой проход и оценка ошибки</b>    
</center> 


__Прямое распространение сигналов__

* Модель нейронной сети это иерархия (она может быть простой и очень сложной) связанных (последовательно применяемых) функций слоев:
    * т.е. модель сети $f_L$ может быть представленна как суперпозиция из $L$ слоев $h^i\text{, }i \in \{1, \ldots, L\}$, каждый из которых параметризуется своими весами $w_i$:
$$f_L(\pmb{x}, \pmb{\theta})=f_L(\pmb{x}, \pmb{w}_1, \ldots, \pmb{w}_L )=h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$$

<center> 
<img src="./img/ann_11.png" alt="многослойный перцептрон с двумя скрытыми слоями" style="width: 600px;"/><br/>
    <b><em class="ex"></em> пример модели сети: многослойный перцептрон с двумя скрытыми слоями</b>    
</center> 

* Прямое распространение сигналов по модели (в частности: нейронной сети) реализуется с помощощью __прямого прохода (forward pass)__: входящая информация (вектор $\pmb{x}$) распространяется через сеть $f_L$ с учетом весов связей $\pmb{\theta}$, расчитывается выходной вектор $\hat{\pmb{y}}=f_L(\pmb{x}, \pmb{\theta})$ .
    * Каждый слой нейронной сети - это последовательно применяемая функция слоя, которая рассчитывается при помощи операций с тензорами.

<center> 
<img src="./img/ann_12.png" alt="пример прямого прохода" style="width: 600px;"/><br/>
    <b><em class="ex"></em> пример прямого прохода</b>    
</center> 

* Последовательность операций с тензормаи используется для расчета результата:
    * Прямой проход (forward pass): входящая информация (вектор $\pmb{x}$) распространяется через сеть с учетом весов связей, расчитывается выходной вектор $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})= h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$
    * Оценки ошибки $E(\hat{\pmb{y}}, \pmb{y})$ на множестве правильных ответов: $\pmb{y}$.

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 600px;"/><br/>
    <b>Прямой проход и оценка ошибки</b>    
</center> 

<em class="qs"></em> Почему для реализации модели ИНС используют тензоры?

Рассмотрим примеры моделей (веса):
* 1 персептрон (4 входа + константа(смещение, baias) ): 
```python
inputs = torch.tensor([1.0, 2.0, 3.0, 2.5])
weights = torch.tensor([0.2, 0.8, -0.5, 1.0, 2.0])```
* 1 слой персептронов из 3 нейронов (ось 0), каждый с 4 входами и константой (ось 1):
```python
inputs = torch.tensor([1.0, 2.0, 3.0, 2.5])
weights = torch.tensor([[0.2, 0.8, -0.5, 1.0, 2.0],
                        [0.5, -0.91, 0.26, -0.5, 3.0],
                        [-0.26, -0.27, 0.17, 0.87, 0.5]])
```
* 2 слоя персептронов из 3 нйронов и из 2 нейронов
```python
inputs = torch.tensor([1.0, 2.0, 3.0, 2.5])
weights_layer1 = torch.tensor([[0.2, 0.8, -0.5, 1.0, 2.0],
                               [0.5, -0.91, 0.26, -0.5, 3.0],
                               [-0.26, -0.27, 0.17, 0.87, 0.5]])
# выход слоя 1 - вход слоя 2
weights2_layer2 = torch.tensor([[0.8, 0.5, 1.1, 2.0],
                                [0.4, 0.26, -0.4, 3.0],
                                [-0.2, 0.27, 0.17, 0.5]])
```
* набор входных вектров (batch):
```python
inputs = torch.tensor([[1.0, 2.0, 3.0, 2.5],
                       [-1.1, 3.0, 2.1, 0.8],
                       [-2.0, 1.3, 0.1, -1.8],
                       [-1.3, 3.2, 1.1, 0.6]])
weights = torch.tensor([[0.2, 0.8, -0.5, 1.0, 2.0],
           [0.5, -0.91, 0.26, -0.5, 3.0],
           [-0.26, -0.27, 0.17, 0.87, 0.5]])
```

In [60]:
import torch

In [70]:
# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):

# вариант исходных данны #1 (2й признак всегда равен 0):
# X = torch.tensor([[1., 0.],
#                   [2., 0.],
#                   [3., 0.],
#                   [4., 0.]], dtype=torch.float32) # Size([4, 2])

# вариант исходных данны #2 (2й признак используется и существенно больше 1го):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

print(f'Матрица X: \n{X}')
print(f'X.size = {X.size()}') 

# истинное значение весов (используется только для получения обучающих правильных ответов):

# вариант #1:
# w_ans = torch.tensor([2., 0.], dtype=torch.float32)

# вариант #2:
w_ans = torch.tensor([2., 1.], dtype=torch.float32)

# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

Матрица X: 
tensor([[ 1., 40.],
        [ 2., 30.],
        [ 3., 20.],
        [ 4., 10.]])
X.size = torch.Size([4, 2])
w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])


In [71]:
# model (модель, в нашем случае: линейная регрессия)

# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=False)

print(f'w:\n{w}')

# прямое распространение (тут фактически описывается модель):
def forward(X):
    return X @ w # Size([4])

w:
tensor([0., 0.])


---
## Обучение модели нейронной сети, алгоритм обратного распространения ошибки <a class="anchor" id="обратное-распространение"></a>
* [к оглавлению](#разделы)

### Проблема обучения модели нейронной сети <a class="anchor" id="проблема-обучения"></a>
* [к оглавлению](#разделы)

__Проблема обучения модели нейронной сети: общий взгляд__

* <em class="nt"></em> __основная проблема__ это не применение модели к входным данным $\pmb{x}$ и оцнка ошибки на правильных ответах $\pmb{y}$, а __обучение модели__ (опредление наилучших параметров модели $\pmb{\theta}$). 
     * В случае нейронной сети обучение сводится к поиску весов слоев сети $\pmb{\theta}=(\pmb{w}_1, \ldots, \pmb{w}_L)$, которые в совокупности являются параметрами модели $\pmb{\theta}$.

* Формально: цель обучения - найти оптимальное значение параметров $\theta^{*}$, минимизирующих ошибку на обучающией выборке $D$: 
$$\theta^{*} = \arg \underset{\pmb{\theta}}{\min} \ E(\pmb{\theta}) = \arg \underset{\pmb{\theta}}{\min} \ \sum_{(\pmb{x}, \pmb{y}) \in D} E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$$
* Т.е. задача обучения сводится к задаче оптимизации.
    * <em class="nt"></em> На самом деле __все сложнее__: хороший результат на $D$ может плохо обобщаться (модель может давать низкое качество на другой выборке из той же генеральной совокупности) - __проблема переобучения__.

<center> 
<img src="./img/main_cycle_p1_v1.png" alt="Приниципиальная логика обучения нейронной сети" style="width: 800px;"/><br/>
    <b>Приниципиальная логика обучения нейронной сети</b>    
</center>

__Прямой проход и оценка ошибки__

* __Прямой проход__ (forward pass): входящая информация (вектор $\pmb{x}$) распространяется через сеть с учетом весов связей, расчитывается выходной вектор $\hat{\pmb{y}} = f_L(\pmb{x}, \pmb{\theta})= h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$

<center> 
<img src="./img/ann_12.png" alt="пример прямого прохода" style="width: 300px;"/><br/>
    <b><em class="ex"></em> пример прямого прохода</b>    
</center> 

* __Оценки ошибки__ $E(\hat{\pmb{y}}, \pmb{y})$ на множестве правильных ответов: $\pmb{y}$.

<center> 
<img src="./img/main_cycle_p2_v2.png" alt="Прямой проход и оценка ошибки" style="width: 400px;"/><br/>
    <b>Прямой проход и оценка ошибки в общей логике обучения нейронной сети</b>    
</center> 

__Задача оптимизации__

* Задача: корректировка весов сети (параметров модели $\pmb{\theta}$) на основе информации об ошибке на обучающих примерах $E(\hat{\pmb{y}}, \pmb{y})$.
    * Решение: использовать методы оптимизации, основанные на __методе градиентного спуска__.
    

* __Метод градиентныого спуска__ - метод нахождения локального экстремума (минимума или максимума) функции с помощью движения вдоль градиента. В нашем случае шаг метода градиентного спуска выглядит следующим образом:
$$\pmb{\theta}_t = \pmb{\theta}_{t-1}-\gamma\nabla_\theta E(\pmb{\theta}_{t-1}) = \pmb{\theta}_{t-1}-\gamma \sum_{(\pmb{x}, \pmb{y}) \in D} \nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$$

* <em class="nt"></em> Выполнение на каждом шаге градиентого спуска суммирование по всем $(\pmb{x}, \pmb{y}) \in D$ __обычно слшиком неэффективно__


* Для выпуклых функций __задача локальной оптимизации__ - найти локальный минимум (максимум) автоматически превращается в __задачу глобальной оптимизации__ - найти точку, в которой достигается наименьшее (наибольшее) значение функции, то есть самую низкую (высокую) точку среди всех.
* Оптимизировать веса одного перцептрона - выпуклая задача, но __для большой нейронной сети  целевая функция не является выпуклой__.

<center> 
<img src="./img/ann_15.png" alt="Прямой проход и оценка ошибки" style="width: 500px;"/><br/>
    <b>Пример работы градиентного спуска для функции двух переменных</b>    
</center>

* У нейронных сетей функция ошибки может задавать __очень сложный ландшафт__ с огромным числом локальных максимумов и минимумов. Это свойство необходимо для обеспечения __выразительности нейронных сетей__, позволяющей им решать так много разных задач.


* <em class="nt"></em> для использования методов, основанных на методе градиентного спуска __необходимо знать градиент функции потерь по параметрам модели__: $\nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$. Этот градиент определяет вектор ("направление") изменения параметров.

In [72]:
# model (модель, в нашем случае: линейная регрессия)

# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=False)

print(f'w:\n{w}')

# прямое распространение:
def forward(X):
    return X @ w # Size([4])

# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
def loss(y, y_pred):
    return ((y_pred - y)**2).mean() # Size([])

# градиент: 
# рассчитан аналитически по модели и функции потерь:
# J = MSE = 1/N * (w*x - y)**2
# dJ/dw = 1/N * 2 * (w*x - y) * x
def gradient(x, y, y_pred):   
#     print(f'''y = {y},
#     y_pred = {y_pred},
#     (2* (y_pred - y)).unsqueeze(1) = {(2* (y_pred - y)).unsqueeze(1)},
#     x = {x},
#     ((2* (y_pred - y)).unsqueeze(1) * x) = {((2* (y_pred - y)).unsqueeze(1) * x)},
#     ((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0) = {((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0)}''')
    return ((2* (y_pred - y)).unsqueeze(1) * x).mean(dim=0)    

w:
tensor([0., 0.])


In [73]:
learning_rate = 0.0013

# predict = forward pass
y_pred = forward(X)

# loss
l = loss(Y, y_pred)

# calculate gradients
dw = gradient(X, Y, y_pred)

# update weights
w -= learning_rate * dw

In [74]:
# Training

# вариант #1:
# learning_rate = 0.05
# n_iters = 20 + 1

# вариант #2:
learning_rate = 0.0013
n_iters = 1000 + 1

# основной цикл:
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
    
    # calculate gradients
    dw = gradient(X, Y, y_pred)

    # update weights
    w -= learning_rate * dw

    if epoch % 5 == 0:
        print(f'epoch {epoch}: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')

epoch 0: w = tensor([0.04740, 0.08853]), y_pred = tensor([88.56900, 66.63800, 44.70700, 22.77600]), loss = 901.66833496
gradient = tensor([  93.53500, 1631.90002])
epoch 5: w = tensor([0.27269, 1.96004]), y_pred = tensor([9.90674, 7.60272, 5.29869, 2.99467]), loss = 595.12457275
gradient = tensor([ -103.50652, -1319.86426])
epoch 10: w = tensor([0.26484, 0.43254]), y_pred = tensor([73.32084, 55.41520, 37.50956, 19.60392]), loss = 393.66198730
gradient = tensor([  57.54780, 1070.75989])
epoch 15: w = tensor([0.43660, 1.65862]), y_pred = tensor([21.67870, 16.68858, 11.69846,  6.70834]), loss = 261.16906738
gradient = tensor([ -71.50771, -865.57104])
epoch 20: w = tensor([0.45417, 0.65508]), y_pred = tensor([63.23888, 48.05265, 32.86644, 17.68021]), loss = 173.95428467
gradient = tensor([ 34.33218, 702.63275])
epoch 25: w = tensor([0.58835, 1.45793]), y_pred = tensor([29.32578, 22.64829, 15.97079,  9.29329]), loss = 116.47224426
gradient = tensor([ -50.14605, -567.58569])
epoch 30: w = te

epoch 815: w = tensor([1.99973, 1.00002]), y_pred = tensor([42.00047, 34.00000, 25.99954, 17.99908]), loss = 0.00000032
gradient = tensor([-0.00229,  0.00019])
epoch 820: w = tensor([1.99974, 1.00002]), y_pred = tensor([42.00044, 34.00000, 25.99957, 17.99913]), loss = 0.00000029
gradient = tensor([-0.00217,  0.00012])
epoch 825: w = tensor([1.99975, 1.00002]), y_pred = tensor([42.00042, 34.00000, 25.99959, 17.99918]), loss = 0.00000026
gradient = tensor([-0.00205,  0.00023])
epoch 830: w = tensor([1.99977, 1.00002]), y_pred = tensor([42.00040, 34.00000, 25.99961, 17.99922]), loss = 0.00000023
gradient = tensor([-0.00195,  0.00019])
epoch 835: w = tensor([1.99978, 1.00001]), y_pred = tensor([42.00037, 34.00000, 25.99963, 17.99926]), loss = 0.00000021
gradient = tensor([-1.86062e-03, -8.58307e-05])
epoch 840: w = tensor([1.99979, 1.00001]), y_pred = tensor([42.00036, 34.00000, 25.99965, 17.99930]), loss = 0.00000019
gradient = tensor([-0.00174,  0.00025])
epoch 845: w = tensor([1.99980, 

### Проблема поиска градиента <a class="anchor" id="проблема-поиска"></a>
* [к оглавлению](#разделы)

* <em class="qs"></em> Проблема: как найти градиент для нейронной сети: $\nabla_\theta E(f_L(\pmb{x}, \pmb{\theta}), \pmb{y})$?

<center> 
<img src="./img/main_cycle_p3_v1.png" alt="Прямой проход и оценка ошибки" style="width: 400px;"/><br/>
    <b>Проблема поиска градиента в общей логике обучения нейронной сети</b>    
</center> 

* Для решения этой задачи и используется __алгоритм обратного распространения ошибки__ (backpropagation). Суть алгоритма:
    * рассчитывается ошибка между выходным вектором сети $\hat{\pmb{y}}$ и правильным ответом обучающего примера $\pmb{y}$
    * ошибка распростаняется от результата к источнику (в обратную сторону) для корректировки весов

<center> 
<img src="./img/ann_13.png" alt="Пример обратного распространения ошибки" style="width: 400px;"/><br/>
    <b><em class="ex"></em>Пример обратного распространения ошибки</b>    
</center>

__Рассчет градиента суперпозиции двух функций нескольких переменных__

* Сначала рассмотрим подзадачу: как рассчитать градиент для $f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L )=h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L)$? 

* Для этого нам нужно будет __рассчитывать градиент суперпозиции (сложной функции)__ состоящей из последовательного применения функций слоев $h^i$.

* Вспомним, как рассчитать производную (градиент) суперпозиции нескольких функций.
    * Пусть $z=f(y)$, $y=g(x)$
    * Тогда производная суперпозиции функций (правило дифференцирования сложной функции (chain rule)): $\frac{\mathrm{d} z}{\mathrm{d} x}=\frac{\mathrm{d} z}{\mathrm{d} y}\frac{\mathrm{d} y}{\mathrm{d} x}$
    * Если $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$, а $\mathbf{z} \in \mathbb{R}$, то: $\frac{\partial z }{\partial x_i} = \sum_j \frac{\partial z}{\partial y_j} \frac{\partial y_j}{\partial x_i}$

<center> 
    
__Примеры рассчета градиента суперпозиции двух функций нескольких переменных:__

<img src="./img/ann_18.png" alt="Примеры иерархий в нейронных сетях" style="width: 500px;"/>
</center>
Т.е. нам нужны градиенты по всем возможным путям (рассмотренным в обработном порядке) завимиостей переменных.

Запись этой же задачи в векторной нотации: 
* $\frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \nabla_x (z)= \begin{pmatrix}
    \dfrac{\partial z}{\partial x_1} \\ \cdots \\ \dfrac{\partial z}{\partial x_n} \end{pmatrix}=\left ( \frac{\mathrm{d} \mathbf{y}}{\mathrm{d} \mathbf{x}} \right )^T \cdot \nabla_y (z) = J(\mathbf{y}(\mathbf{x}))^T \cdot \nabla_y (z)= J(\mathbf{y}(\mathbf{x}))^T \cdot \begin{pmatrix}
    \dfrac{\partial z}{\partial y_1} \\ \cdots \\ \dfrac{\partial z}{\partial y_m} \end{pmatrix}$    
* Где $J$ это Якобиан: $$J(\mathbf{y}(\mathbf{x})) = \begin{pmatrix}
    \dfrac{\partial y_1}{\partial x_1} & \cdots & \dfrac{\partial y_1}{\partial x_n}\\
    \vdots & \ddots & \vdots\\
    \dfrac{\partial y_m}{\partial x_1} & \cdots & \dfrac{\partial y_m}{\partial x_n} \end{pmatrix} $$ 

__Задача поиска градиента: $\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})$__

* Перейдем от $f_L$ к последовательному рассчету функций слоев $h^i$:
$$\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_{\mathbf{w}_i} E(f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L ), \mathbf{y})=\nabla_{\mathbf{w}_i} E(h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L), \mathbf{y})$$

* Обозначим через $\mathbf{a}^l$ результат рассчета функции активации на слое $l$: $\mathbf{a}^l=h^l(\mathbf{x}_l,\mathbf{w}_l)$. Тогда: $\mathbf{x}_{l+1}=\mathbf{a}_l$ (вход следущего слоя является результатом рассчета функции активации предыдущего слоя)

* Тогда можно записать: $\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_\theta E(\mathbf{a}^L, \mathbf{y})$. Функция потерь $E(\mathbf{a}^L, \mathbf{y})$ зависит от $\mathbf{a}^L$, $\mathbf{a}^L$ от $\mathbf{a}^{L-1}$, ..., $\mathbf{a}^{l+1}$ от $\mathbf{a}^{l}$

* Исходя из этого представления можно градиенты весов $l$-го слоя можно записать как: 
$$\dfrac{\partial E}{\partial \mathbf{w}_l}=\color{blue}{ \dfrac{\partial E}{\partial \mathbf{a}_L} \cdot \dfrac{\partial \mathbf{a}_L}{\partial \mathbf{a}_{L-1}} \cdot \cdots \cdot \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}}} \cdot \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$$

* Произведение всех сомножетелей кроме последнего является градиентом функции потерь по результатам рассчета функции активации слоя $l$:
$$\color{blue} {\dfrac{\partial E}{\partial \mathbf{a}_L} \cdot \dfrac{\partial \mathbf{a}_L}{\partial \mathbf{a}_{L-1}} \cdot \cdots \cdot \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}}} = \color{blue} {\dfrac{\partial E}{\partial \mathbf{a}_l}}$$

* Тогда:
$$\dfrac{\partial E}{\partial \mathbf{w}_l}=\left ( \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$$ для рассчета $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ нам нужен только якобиан функции активации $l$-го слоя по параметрам слоя $\mathbf{w}_{l}$. 

* Градиент функции потерь по результатам рассчета функции активации слоя  $l$ может быть рассчитан рекурсивно по результатам слоя $l$, собственно тут и происходит __обратное распространение__:
$\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}=\left ( \dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{a}_{l}} \right )^T \cdot \dfrac{\partial E}{\partial \mathbf{a}_{l+1}}=\left ( \color{magenta}{\dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{x}_{l+1}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l+1}}}$ для рассчета $\color{magenta}{\dfrac{\partial \mathbf{a}_{l+1}}{\partial \mathbf{x}_{l+1}}}$ нам нужен только якобиан функции активации $l+1$-го слоя по входным значениям слоя $\mathbf{x}_{l+1}$

* Т.е. чтобы проводить обратное распространение ошибки, нам на каждом слое (например $l$-м) нужно рассчитывать два якобиана:
    * якобиан функции активации $l$-го слоя по параметрам слоя $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ - он позволит рассчитать градиент $\dfrac{\partial E}{\partial \mathbf{w}_l}$ и сделать очередной шаг градиентного спуска для параметров этого слоя: $\mathbf{w}_l^{t+1} = \mathbf{w}_l^{t}-\gamma\nabla_{w_l} E(\mathbf{w}^{t})=\mathbf{w}_l^{t}-\gamma \dfrac{\partial E(\mathbf{w}^{t})}{\partial \mathbf{w}_l}$
    * якобиан функции активации $l$-го слоя по входным значениям слоя: $\color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}}$ - он позволит распространить ошибку на низлежащие слои.

Такми образом при обучении нам нужна только __очень локальная информация, содержащаяся в самом слое__. Т.е.:
* __нет нобходимости знать как устроены сосоедние слои__: между слоями __очень простой интерфейс__ 
* т.е. __можно создать модульную архитектуру для слоев нейронной сети__: каждый модуль рассчитывает значение функции активации на основе выходов на прямом проходе и распространяет ошибку пришедшую на выходы на обратном проходе; все модули станадртным образом стыкуются друг с другом
* при модульной архитектуре граф нейронной сети может быть очень сложным, но его __рассчет выполняется по одной простой и универсальной схеме__
* внутри __модули могут быть сложно устроены, это никак не меняет логику остальных модулей и всего процесса обучения__, главное чтобы модуль корректно выполнял прямой и обратный проход.

-----------

##  Дифференцируемое программирование и реализация обратного распространения ошибки <a class="anchor" id="дифференцируемое"></a>
* [к оглавлению](#разделы)

__Почему Tensor *Flow*?__

<em class="qs"></em> Как реализовать __алгоритм обратного распространения ошибки__ удобно для использования в задачах моделирования ИНС?

Основная абстракция TensorFlow, PyTorch и других аналогичных библиотеках - __граф потока вычислений__.

* Рассматриваемые библиотеки обычно:
    1. задают __граф потока вычислений__ (формирует объект отложенных вычислений)
    2. запускают __процедуру выполненния отложенных вычислений__ и получает __результаты__ вычислений (в т.ч. ошибку модели).
* Возможность в явном виде работать с графом потока вычислений дает большое приемущество для __автоматического решения задачи обратного распространения ошибки__, являющейся составляющей адачей обучения модели ИНС.

<center> 
<img src="./img/ker_6.png" alt="Принцип устройства графа потока вычислений в TensorFlow" style="width: 400px;"/><br/>
    <b>Принцип устройства графа потока вычислений в TensorFlow</b>    
</center>

* Нейронная сеть это иерархия (она может быть простой и очень сложной) связанных (последовательно применяемых) функций слоев ИНС. Модель сети $f_L$ может быть представленна как суперпозиция из $L$ слоев $h^i\text{, }i \in \{1, \ldots, L\}$, каждый из которых параметризуется своими весами $w_i$:
$$f_L(\pmb{x}, \pmb{\theta})=f_L(\pmb{x}, \pmb{w}_1, \ldots, \pmb{w}_L )=h^L(h^{L-1}(\ldots h^1(\pmb{x}, \pmb{w}_1), \ldots, \pmb{w}_{L-1}),\pmb{w}_L)$$

* Вычисление функций слоев и взаимосвязи между слоями формируют граф потока вычислений в библиотеке моделирования ИНС.

<center> 
<img src="./img/ann_16.png" alt="Примеры иерархий в нейронных сетях" style="width: 400px;"/><br/>
    <b>Примеры иерархий в нейронных сетях</b>    
</center>


* По сути, ИНС это композиция модулей, представляющих собой слои нейронной сети:
    * если сеть прямого распространения (feedforward), то все просто
    * если сеть является направленным ациклическим графом, то существует правильный порядок применения функций
    * в случае, если есть циклы, образующие рекуррентные связи, то существуют специальные подходы (будут рассмотрены позднее)


* На обратном проходе (при обратном распространении ошибки) нам необходимо __дифференциировать сложную функцию__ многослойной ИНС
$$\nabla_\theta E(f_L(\mathbf{x}, \mathbf{\theta}), \mathbf{y})=\nabla_{\mathbf{w}_i} E(f_L(\mathbf{x}, \mathbf{w}_1, \ldots, \mathbf{w}_L ), \mathbf{y})=\nabla_{\mathbf{w}_i} E(h^L(h^{L-1}(\ldots h^1(\mathbf{x}, \mathbf{w}_1), \ldots, \mathbf{w}_{L-1}),\mathbf{w}_L), \mathbf{y})$$
* алгоритм обратного распространения ошибки позволяет свести эту задачу к дифференциированию составляющих функций, но для этого необходимо __храниить информацию о виде и взаимосвязях функций задействованных в расчете модели ИНС__, именно эта информация и хранится в графе потока вычислений. Это позволяет организовать __автоматическое дифференциирование__ сложной функци многослойной ИНС.

__Дифференциируемое программирование__

<em class="df"></em> __Дифференциируемое программирование__ (differentiable programming) - парадигма программирования при которой программа (функция рассчета значения) может быть продифференциирована в любой точке, обычно с помощью __автоматического диффиренциирования__. 

Это свойство позволяет использовать к программе __методы оптимизации основанные на рассчете градиента__, обычно - __методы градиентного спуска__.

Дифференциируемое программирование используется в:
* глубоком обучении
* глубоком обучении комбинированном с физическими моделями в робототехнике
* специализированных методах трассировки лучей
* обработке изображений

Большинство фреймоврков для дифференциируемого программирования использует граф потока вычислений определяющий выполнение программы и ее структуры данных.

Основные классы фреймворков для дифференциируемого программирования:
* __статические__ - они компилируют граф потока вычислений. Типичные представители: TensorFlow, Theano и др. Плюсы и минусы
    * <em class="pl"></em> могут использовать оптимизацию при компиляции
    * <em class="pl"></em> легче масштабирются на большие системы
    * <em class="mn"></em> статичность ограничевает интерактивность
    * <em class="mn"></em> многие программы не могут реализовываться легко (в частности: циклы, рекурсия)
* __динамические__ - динамически исполняют граф потока вычислений. Используют перегрузку операторов для записи. Типичные представители: PyTorch, AutoGradrFlow. Плюсы и минусы:
    * <em class="pl"></em> более простая и понятная запись программы
    * <em class="mn"></em> накладные расходы интерпретатора
    * <em class="mn"></em> невозможно использовать оптимизацию компилятора
    * <em class="mn"></em> хуже масштабируемость
* статическая на основе разбора промежуточного представления синтаксического разбора исходной программы. Пример фрэймоврк Zygote (язык программирования Julia).

__Прямой проход__:
* Модули из графа обходятся один за одним начиная с узла входных данных и далее по мере готовности всех необходимых входных данных для очередного модуля, который еще не был обойден
* Рассчет функций активации для каждого модуля по входным данным: $a_l=h_l(x_l, w_l)$
* Промежуточные значения кэшируются, чтобы не рассчитывать их повторно (в сложном графе сети и при обратном проходе)
* Выходы одних модулей становятся входами других модулей: $x_{l+1}=a_l$
* Последним модулем рассчитывается сумма потерь для входных данных
<center> 
    
__Прямой и обратный проход процедуры обучения многослойной ИНС:__

<img src="./img/ann_19.png" alt="Прямой и обратный проход " style="width: 300px;"/>
</center>


__Обратный проход__:
* Сначала должен быть произведен прямой проход. На входе обратного прохода известна сумма потерь.
* Строится обратный порядок обхода графа зависимостей модулей.
* Модули из графа обходятся один за одним начиная с узла рассчета функции потерь и далее по мере готовности всех необходимых входных данных для очередного модуля, который еще не был обойден 
* Для каждого модуля рассчитыватся якобиан функции активации по параметрам слоя $\color{red}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}}$ и якобиан функции активации по входным значениям слоя: $\color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}}$ 
* По пришедшму в модуль градиенту ошибки (полученному из модулей использовавших результаты данного модуля на прямом проходе) $\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$ рассчитывается:
    * Градиент для шага градиентного спуска по параметрам модуля $w_l$: $\dfrac{\partial E}{\partial \mathbf{w}_l}=\left ( \color{red}{ \dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{w}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_l}}$
    * Градиент ошбки, который передается в модули, поставившие данные в этот модуль во время прямого прохода: $\color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l-1}}}=\left ( \color{magenta}{\dfrac{\partial \mathbf{a}_{l}}{\partial \mathbf{x}_{l}}} \right )^T \cdot \color{blue}{\dfrac{\partial E}{\partial \mathbf{a}_{l}}}$ 

<center> 
    
__Пример: прямой проход, шаг 1__

<img src="./img/bp_2.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: прямой проход, шаг 2__

<img src="./img/bp_3.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: прямой проход, шаг 3__

<img src="./img/bp_4.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: обратный проход, шаг 1__

<img src="./img/bp_5.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Пример: обратный проход, шаг 2__

<img src="./img/bp_6.png" alt="Пример" style="width: 700px;"/>
</center>

<center> 
    
__Производные популярных функций активации__

<img src="./img/ann_17.png" alt="Пример" style="width: 500px;"/>
</center>


<center> 
    
__Пример: обратный проход, шаг 3__

<img src="./img/bp_7.png" alt="Пример" style="width: 700px;"/>
</center>

:
##  Автоматическое дифференциирование в PyTorch <a class="anchor" id="автоматическое-PyTorch"></a>
* [к оглавлению](#разделы)

In [79]:
# The autograd package provides automatic differentiation 
# for all operations on Tensors

# requires_grad = True -> tracks all operations on the tensor. 
x = torch.randn(3, requires_grad=True)
y = x + 2

# y was created as a result of an operation, so it has a grad_fn attribute.
# grad_fn: references a Function that has created the Tensor
print(x) # created by the user -> grad_fn is None
print(y)
print(y.grad_fn)

tensor([-0.08521,  0.25389, -0.85220], requires_grad=True)
tensor([1.91479, 2.25389, 1.14780], grad_fn=<AddBackward0>)


In [80]:
# Do more operations on y
z = y * y * 3
print(z)
z = z.mean()
print(z)

tensor([10.99930, 15.24001,  3.95233], grad_fn=<MulBackward0>)
tensor(10.06388, grad_fn=<MeanBackward0>)


In [81]:
# Let's compute the gradients with backpropagation
# When we finish our computation we can call .backward() and have all the gradients computed automatically.
# The gradient for this tensor will be accumulated into .grad attribute.
# It is the partial derivate of the function w.r.t. the tensor

z.backward()
print(x.grad) # dz/dx

# Generally speaking, torch.autograd is an engine for computing vector-Jacobian product
# It computes partial derivates while applying the chain rule

tensor([3.82959, 4.50777, 2.29560])


---

__Примеры рассчета градиента суперпозиции двух функций нескольких переменных:__

<img src="./img/ann_18.png" alt="Примеры иерархий в нейронных сетях" style="width: 500px;"/>
</center>
Т.е. нам нужны градиенты по всем возможным путям (рассмотренным в обработном порядке) завимиостей переменных.

Запись этой же задачи в векторной нотации: 
* $\frac{\mathrm{d} z}{\mathrm{d} \mathbf{x}} = \nabla_x (z)= \begin{pmatrix}
    \dfrac{\partial z}{\partial x_1} \\ \cdots \\ \dfrac{\partial z}{\partial x_n} \end{pmatrix}=\left ( \frac{\mathrm{d} \mathbf{y}}{\mathrm{d} \mathbf{x}} \right )^T \cdot \nabla_y (z) = J(\mathbf{y}(\mathbf{x}))^T \cdot \nabla_y (z)= J(\mathbf{y}(\mathbf{x}))^T \cdot \begin{pmatrix}
    \dfrac{\partial z}{\partial y_1} \\ \cdots \\ \dfrac{\partial z}{\partial y_m} \end{pmatrix}$    
* Где $J$ это Якобиан: $$J(\mathbf{y}(\mathbf{x})) = \begin{pmatrix}
    \dfrac{\partial y_1}{\partial x_1} & \cdots & \dfrac{\partial y_1}{\partial x_n}\\
    \vdots & \ddots & \vdots\\
    \dfrac{\partial y_m}{\partial x_1} & \cdots & \dfrac{\partial y_m}{\partial x_n} \end{pmatrix} $$ 

In [82]:
# Model with non-scalar output:
# If a Tensor is non-scalar (more than 1 elements), we need to specify arguments for backward() 
# specify a gradient argument that is a tensor of matching shape.
# needed for vector-Jacobian product

x = torch.randn(3, requires_grad=True)

y = x * 2
for _ in range(10):
    y = y * 2

print(y)
print(y.shape)

tensor([ 2596.08423,  -432.97888, -2130.46240], grad_fn=<MulBackward0>)
torch.Size([3])


In [83]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float32)
y.backward(v)
print(x.grad)

tensor([2.04800e+02, 2.04800e+03, 2.04800e-01])


--- 
Stop a tensor from tracking history:
For example during our training loop when we want to update our weights
then this update operation should not be part of the gradient computation
* `x.requires_grad_(False)`
* `x.detach()`
* wrap in `with torch.no_grad():`

In [85]:
# .requires_grad_(...) changes an existing flag in-place.

a = torch.randn(2, 2)
print(f'a.requires_grad = {a.requires_grad}')

b = ((a * 3) / (a - 1))
print(f'b.grad_fn = {b.grad_fn}')
      
a.requires_grad_(True)
print(f'a.requires_grad = {a.requires_grad}')

b = (a * a).sum()
print(f'b.grad_fn = {b.grad_fn}')

a.requires_grad = False
b.grad_fn = None
a.requires_grad = True
b.grad_fn = <SumBackward0 object at 0x00000221B56D26C8>


In [87]:
# .detach(): get a new Tensor with the same content but no gradient computation:
a = torch.randn(2, 2, requires_grad=True)
print(a.requires_grad)

b = a.detach()
print(b.requires_grad)

True
False


In [88]:
# wrap in 'with torch.no_grad():'
a = torch.randn(2, 2, requires_grad=True)
print(a.requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
False


------

In [89]:
# backward() accumulates the gradient for this tensor into .grad attribute.
# !!! We need to be careful during optimization !!!
# Use .zero_() to empty the gradients before a new optimization step!

weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    # just a dummy example
    # 'forward pass'
    model_output = (weights*3).sum()
    
    
    model_output.backward()
    
    print(weights.grad)

    # optimize model, i.e. adjust weights...
    with torch.no_grad():
        weights -= 0.1 * weights.grad

    # this is important! It affects the final weights & output
    weights.grad.zero_()

print(weights)
print(model_output)

# Optimizer has zero_grad() method
# optimizer = torch.optim.SGD([weights], lr=0.1)
# During training:
# optimizer.step()
# optimizer.zero_grad()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([0.10000, 0.10000, 0.10000, 0.10000], requires_grad=True)
tensor(4.80000, grad_fn=<SumBackward0>)


-----

Автоматическое выполнение обратного прохода с помощью `l.backward()`:

In [90]:
# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):

X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])


# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

# model (модель, в нашем случае: линейная регрессия)

# изначальное значение весов w
#!!! requires_grad=True
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True)

# прямое распространение:
def forward(X):
    return X @ w # Size([4])

# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
def loss(y, y_pred):
    return ((y_pred - y)**2).mean() # Size([])


# Training
learning_rate = 0.0013
n_iters = 1000 + 1

# основной цикл:
for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
    
    #!!! backward pass        
    # calculate gradients = backward pass
    l.backward()
    
    # update weights
    #w.data = w.data - learning_rate * w.grad
    with torch.no_grad():
        w -= learning_rate * w.grad
    
    # zero the gradients after updating
    w.grad.zero_()    
    
    if epoch % 5 == 0:
        print(f'epoch {epoch}: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')
    

w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])
epoch 0: w = tensor([0.16900, 2.21000], requires_grad=True), y_pred = tensor([0., 0., 0., 0.], grad_fn=<MvBackward>), loss = 980.00000000
gradient = tensor([-0.00032, -0.00026])
epoch 5: w = tensor([0.13813, 0.24422], requires_grad=True), y_pred = tensor([81.70274, 61.57523, 41.44772, 21.32021], grad_fn=<MvBackward>), loss = 646.58898926
gradient = tensor([-0.00032, -0.00026])
epoch 10: w = tensor([0.33966, 1.82453], requires_grad=True), y_pred = tensor([15.22920, 11.70164,  8.17407,  4.64651], grad_fn=<MvBackward>), loss = 427.49307251
gradient = tensor([-0.00032, -0.00026])
epoch 15: w = tensor([0.34364, 0.53338], requires_grad=True), y_pred = tensor([68.78386, 52.09385, 35.40385, 18.71384], grad_fn=<MvBackward>), loss = 283.42614746
gradient = tensor([-0.00032, -0.00026])
epoch 20: w = tensor([0.49880, 1.56850], requires_grad=True), y_pred = tensor([25.13912, 19.37721, 13.61531,  7.85340], grad_fn=<MvBackward>), loss 

gradient = tensor([-0.00032, -0.00026])
epoch 550: w = tensor([1.99511, 1.00033], requires_grad=True), y_pred = tensor([42.00831, 34.00005, 25.99179, 17.98354], grad_fn=<MvBackward>), loss = 0.00010187
gradient = tensor([-0.00032, -0.00026])
epoch 555: w = tensor([1.99537, 1.00031], requires_grad=True), y_pred = tensor([42.00787, 34.00005, 25.99223, 17.98441], grad_fn=<MvBackward>), loss = 0.00009140
gradient = tensor([-0.00032, -0.00026])
epoch 560: w = tensor([1.99561, 1.00029], requires_grad=True), y_pred = tensor([42.00747, 34.00006, 25.99264, 17.98523], grad_fn=<MvBackward>), loss = 0.00008202
gradient = tensor([-0.00032, -0.00026])
epoch 565: w = tensor([1.99584, 1.00028], requires_grad=True), y_pred = tensor([42.00705, 34.00004, 25.99302, 17.98600], grad_fn=<MvBackward>), loss = 0.00007359
gradient = tensor([-0.00032, -0.00026])
epoch 570: w = tensor([1.99606, 1.00026], requires_grad=True), y_pred = tensor([42.00670, 34.00005, 25.99340, 17.98675], grad_fn=<MvBackward>), loss = 0

------
Использование встроенного оптимизатора `optimizer = torch.optim.SGD([w], lr=learning_rate)` и функции потерь `loss = nn.MSELoss()`:

In [91]:
import torch
import torch.nn as nn

# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

#--------------------
# 0) Training samples

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

#--------------------
# 1) Design Model: Weights to optimize and forward function

# изначальное значение весов w
w = torch.tensor([0.0, 0.0], dtype=torch.float32, requires_grad=True)


# model (модель, в нашем случае: линейная регрессия)
# прямое распространение:
def forward(X):
    return X @ w # Size([4])

#--------------------
# 2) Define loss and optimizer

# callable function
loss = nn.MSELoss()

# loss = MSE (функция потерь, в нашем слаучае: средняя квадратичная ошибка)
# def loss(y, y_pred):
#     return ((y_pred - y)**2).mean() # Size([])

learning_rate = 0.0013
optimizer = torch.optim.SGD([w], lr=learning_rate)


#--------------------
# 3) Training loop
# основной цикл:
n_iters = 1000 + 1

for epoch in range(n_iters):
    # predict = forward pass
    y_pred = forward(X)

    # loss
    l = loss(Y, y_pred)
         
    # calculate gradients = backward pass
    l.backward()
    
    # update weights
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()    
    
    if epoch % 5 == 0:
        print(f'epoch {epoch}: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')
    

w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])
epoch 0: w = tensor([0.16900, 2.21000], requires_grad=True), y_pred = tensor([0., 0., 0., 0.], grad_fn=<MvBackward>), loss = 980.00000000
gradient = tensor([-0.00032, -0.00026])
epoch 5: w = tensor([0.13813, 0.24422], requires_grad=True), y_pred = tensor([81.70274, 61.57523, 41.44772, 21.32021], grad_fn=<MvBackward>), loss = 646.58898926
gradient = tensor([-0.00032, -0.00026])
epoch 10: w = tensor([0.33966, 1.82453], requires_grad=True), y_pred = tensor([15.22920, 11.70164,  8.17407,  4.64651], grad_fn=<MvBackward>), loss = 427.49307251
gradient = tensor([-0.00032, -0.00026])
epoch 15: w = tensor([0.34364, 0.53338], requires_grad=True), y_pred = tensor([68.78386, 52.09385, 35.40385, 18.71384], grad_fn=<MvBackward>), loss = 283.42614746
gradient = tensor([-0.00032, -0.00026])
epoch 20: w = tensor([0.49880, 1.56850], requires_grad=True), y_pred = tensor([25.13912, 19.37721, 13.61531,  7.85340], grad_fn=<MvBackward>), loss 

gradient = tensor([-0.00032, -0.00026])
epoch 515: w = tensor([1.99285, 1.00048], requires_grad=True), y_pred = tensor([42.01215, 34.00007, 25.98800, 17.97593], grad_fn=<MvBackward>), loss = 0.00021764
gradient = tensor([-0.00032, -0.00026])
epoch 520: w = tensor([1.99323, 1.00045], requires_grad=True), y_pred = tensor([42.01151, 34.00008, 25.98864, 17.97721], grad_fn=<MvBackward>), loss = 0.00019526
gradient = tensor([-0.00032, -0.00026])
epoch 525: w = tensor([1.99359, 1.00043], requires_grad=True), y_pred = tensor([42.01090, 34.00007, 25.98924, 17.97841], grad_fn=<MvBackward>), loss = 0.00017518
gradient = tensor([-0.00032, -0.00026])
epoch 530: w = tensor([1.99392, 1.00041], requires_grad=True), y_pred = tensor([42.01033, 34.00007, 25.98981, 17.97955], grad_fn=<MvBackward>), loss = 0.00015719
gradient = tensor([-0.00032, -0.00026])
epoch 535: w = tensor([1.99425, 1.00039], requires_grad=True), y_pred = tensor([42.00978, 34.00006, 25.99035, 17.98063], grad_fn=<MvBackward>), loss = 0

-----------
Использование модели:

`torch.nn.Linear(in_features, out_features, bias=True)`
Applies a linear transformation to the incoming data: $y = xA^T + b$

Parameters:
* `in_features` – size of each input sample
* `out_features` – size of each output sample
* `bias` – If set to False, the layer will not learn an additive bias. Default: True

In [58]:
# X_test = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32) # torch.tensor([5], dtype=torch.float32)

X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

n_samples, n_features = X.shape

print(n_samples, n_features)

# n_samples, n_features = X_test.shape
# input_size = n_features
# output_size = n_features

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim, bias=False)
    def forward(self, x):
        return self.lin(x)
    
model = LinearRegression(n_features, 1)


print(f'Prediction before training: f(5) = {model(X)}')

4 2
Prediction before training: f(5) = tensor([[ 6.43014],
        [ 4.13109],
        [ 1.83204],
        [-0.46701]], grad_fn=<MmBackward>)


In [59]:
# X_test.shape

In [55]:
import torch
import torch.nn as nn

# Модель линейной регрессии (с несколькими параметрами)
# f = X * w 

#--------------------
# 0) Training samples

# Данные для обучения: 
# принзаки X: рассматривается 4 наблюдения (ось 0) и 2 признака (ось 1):
X = torch.tensor([[1., 40.],
                  [2., 30.],
                  [3., 20.],
                  [4., 10.]], dtype=torch.float32) # Size([4, 2])

print(f'X.shape = {X.shape}')
X_samples, X_features = X.shape

# истинное значение весов (используется только для получения обучающих правильных ответов):
w_ans = torch.tensor([2., 1.], dtype=torch.float32)
# Y - приавильные ответы: 
Y = X @ w_ans
print(f'Y.shape = {Y.shape}')
Y_features = 1

torch.set_printoptions(precision=5) # точность вывода на печать значений тензоров
print(f'w true value = {w_ans}, Y = {Y}')

#--------------------
# 1) Design Model, the model has to implement the forward pass!
# Here we can use a built-in model from PyTorch
input_size = n_features
output_size = n_features

# we can call this model with samples X
# model = nn.Linear(input_size, output_size)

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        # define diferent layers
        self.lin = nn.Linear(input_dim, output_dim, bias=False)
    def forward(self, x):
        return self.lin(x)
    
model = LinearRegression(X_features, Y_features)


print(f'Prediction before training: f({X}) = {model(X)}')


# # model (модель, в нашем случае: линейная регрессия)
# # прямое распространение:
# def forward(X):
#     return X @ w # Size([4])

#--------------------
# 2) Define loss and optimizer

# callable function
criterion  = nn.MSELoss()

learning_rate = 0.0013
optimizer = torch.optim.SGD([w], lr=learning_rate)

#--------------------
# 3) Training loop
# основной цикл:
n_iters = 1000 + 1

for epoch in range(n_iters):
    # Forward pass and loss
    y_predicted = model(X)
    loss = criterion(y_predicted, Y)

    
    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()    
    
    if epoch % 5 == 0:
        print(f'epoch {epoch}: w = {w}, y_pred = {y_pred}, loss = {l:.8f}\ngradient = {dw}')
    

X.shape = torch.Size([4, 2])
Y.shape = torch.Size([4])
w true value = tensor([2., 1.]), Y = tensor([42., 34., 26., 18.])
Prediction before training: f(tensor([[ 1., 40.],
        [ 2., 30.],
        [ 3., 20.],
        [ 4., 10.]])) = tensor([[-0.36370],
        [-0.80204],
        [-1.24038],
        [-1.67872]], grad_fn=<MmBackward>)
epoch 0: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 5: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 10: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 15: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = te

C:\Users\alpha\.conda\envs\pyTorch_1_5v2\lib\site-packages\torch\nn\modules\loss.py:432: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



epoch 415: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 420: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 425: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 430: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 435: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 440

gradient = tensor([ -130., -1700.])
epoch 890: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 895: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 900: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 905: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient = tensor([ -130., -1700.])
epoch 910: w = tensor([1.99996, 1.00000], requires_grad=True), y_pred = tensor([42.00007, 34.00001, 25.99994, 17.99988], grad_fn=<MvBackward>), loss = 0.00000001
gradient =

---
# Спасибо за внимание!

---
### Технический раздел:

 * И Введение в искусственные нейронные сети
     * Базовые понятия и история
 * И Машинное обучение и концепция глубокого обучения
 * И Почему глубокое обучение начало приносить плоды и активно использоваться только после 2010 г?
     * Производительность оборудования
     * Доступность наборов данных и тестов
     * Алгоритмические достижения в области глубокого обучения
         * Улчшенные подходы к регуляризации
         * Улучшенные схемы инициализации весов
         * (повтор) Усовершенствованные методы градиентного супска
         

* Обратное распространение ошибки
 * Оптимизация
     * Стохастический градиентный спуск
     * Усовершенствованные методы градиентного супска
* Введение в PyTorch

<br/> next <em class="qs"></em> qs line 
<br/> next <em class="an"></em> an line 
<br/> next <em class="nt"></em> an line 
<br/> next <em class="df"></em> df line 
<br/> next <em class="ex"></em> ex line 
<br/> next <em class="pl"></em> pl line 
<br/> next <em class="mn"></em> mn line 
<br/> next <em class="plmn"></em> plmn line 
<br/> next <em class="hn"></em> hn line 

* Работа с графом потока вычислений нужна  для того, чтобы решить __задачу обучения многослойной ИНС__. А эта задача требует после получения резуьтатов и оценки ошибки __выполнения обратного прохода__ дающего градиент ошибки для весов (параметров) модели и последующей процедуры оптимизации весов. 

<center> 
<img src="./img/ker_7.png" alt="" style="width: 500px;"/>
<img src="./img/ker_8.png" alt="" style="width: 500px;"/>    
<img src="./img/ker_9.png" alt="" style="width: 500px;"/>        
<img src="./img/ker_10.png" alt="" style="width: 500px;"/>        
<img src="./img/ker_11.png" alt="" style="width: 500px;"/>            
<img src="./img/ker_12.png" alt="" style="width: 500px;"/>                
</center>

In [ ]:
|